In [4]:
import numpy as np
import pandas as pd

from math import sqrt
from sklearn.metrics  import mean_squared_error
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import NearestNeighbors

In [6]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

In [7]:
ratings.shape

(100836, 4)

In [8]:
movies.shape

(9742, 3)

In [10]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [11]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [17]:
dataset = pd.merge(ratings,movies, on='movieId').drop(['timestamp', 'genres'], axis= 1)
print(dataset.shape)
dataset.head()

(100836, 4)


,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [19]:
missing_pivot = dataset.pivot_table(values='rating', index='userId', columns='title')
missing_pivot.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
rate = {}
rows_indexes = {}

for i, row in missing_pivot.iterrows():
    rows = [x for x in range(0,len(missing_pivot.columns))]
    combine = list(zip(row.index, row.values, rows))
    rated = [(x,z) for x,y,z in combine if str(y) != 'nan']
    index = [i[1] for i in rated]
    row_names = [i[0] for i in rated]
    rows_indexes[i] = index
    rate[i] = row_names

rate

{1: ['13th Warrior, The (1999)',
  '20 Dates (1998)',
  'Abyss, The (1989)',
  'Adventures of Robin Hood, The (1938)',
  'Alice in Wonderland (1951)',
  'Alien (1979)',
  'All Quiet on the Western Front (1930)',
  'American Beauty (1999)',
  'American History X (1998)',
  'American Tail, An (1986)',
  'Apocalypse Now (1979)',
  'Austin Powers: International Man of Mystery (1997)',
  'Back to the Future (1985)',
  'Back to the Future Part III (1990)',
  'Bambi (1942)',
  'Basic Instinct (1992)',
  'Batman (1989)',
  'Batman Returns (1992)',
  'Bedknobs and Broomsticks (1971)',
  'Beetlejuice (1988)',
  'Being John Malkovich (1999)',
  'Best Men (1997)',
  'Big (1988)',
  'Big Lebowski, The (1998)',
  'Big Trouble in Little China (1986)',
  'Billy Madison (1995)',
  'Black Cauldron, The (1985)',
  'Blazing Saddles (1974)',
  'Blown Away (1994)',
  'Blues Brothers, The (1980)',
  'Bottle Rocket (1996)',
  'Braveheart (1995)',
  'Canadian Bacon (1995)',
  "Charlotte's Web (1973)",
  'Citiz

In [22]:
pivot_table = dataset.pivot_table(values= 'rating', index= 'userId', columns= 'title').fillna(0)
pivot_table = pivot_table.apply(np.sign)
pivot_table.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
notrated = {}
notrated_indexes = {}

for i,row in pivot_table.itterows():
    rows = [x for x in range(0, len(missing_pivot.columns))]
    